In [1]:
import sys
from pathlib import Path
sys.path.append((Path.cwd().parent.parent).__str__())

In [2]:
from pyqpanda_alg.QAOA import default_circuits
import numpy as np
from pyqpanda3.core import QProg

In [5]:
prog = QProg(4)
qubits = prog.qubits()
circuit = default_circuits.complete_xy_mixer(qubits, np.pi/5)
print(circuit)


          ┌──────────────┐        ┌───────────────┐        ┌───────────────┐ ┌──────────────┐               ┌───────────────┐ >
q_0:  |0>─┤RX(1.57079633)├ ───*── ┤RX(-1.25663706)├ ───*── ┤RX(-1.57079633)├ ┤RX(1.57079633)├ ───*── ────── ┤RX(-1.25663706)├ >
          ├──────────────┤ ┌──┴─┐ ├───────────────┤ ┌──┴─┐ ├───────────────┤ ├──────────────┤    │          ├───────────────┤ >
q_1:  |0>─┤RX(1.57079633)├ ┤CNOT├ ┤RZ(-1.25663706)├ ┤CNOT├ ┤RX(-1.57079633)├ ┤RX(1.57079633)├ ───┼── ───*── ┤RX(-1.25663706)├ >
          ├──────────────┤ └────┘ ├───────────────┤ └────┘ ├───────────────┤ ├──────────────┤ ┌──┴─┐    │   ├───────────────┤ >
q_2:  |0>─┤RX(1.57079633)├ ───*── ┤RX(-1.25663706)├ ───*── ┤RX(-1.57079633)├ ┤RX(1.57079633)├ ┤CNOT├ ───┼── ┤RZ(-1.25663706)├ >
          ├──────────────┤ ┌──┴─┐ ├───────────────┤ ┌──┴─┐ ├───────────────┤ ├──────────────┤ └────┘ ┌──┴─┐ ├───────────────┤ >
q_3:  |0>─┤RX(1.57079633)├ ┤CNOT├ ┤RZ(-1.25663706)├ ┤CNOT├ ┤RX(-1.57079633)├ ┤RX(1.57079633)├ ────── ┤C

In [6]:
from pyqpanda3.core import RX, CPUQVM
for q in qubits:
    prog << RX(q, np.random.random())

machine = CPUQVM()
machine.run(prog, shots=1)
original_result = machine.result().get_prob_dict()

prog << circuit
machine.run(prog, shots=1)
final_result = machine.result().get_prob_dict()

for weight in range(5):
    prob_of_equal_weight_in_origin_result = sum(value for key, value in original_result.items() if key.count('1') == weight)
    prob_of_equal_weight_in_final_result =  sum(value for key, value in final_result.items() if key.count('1') == weight)

    print(f"Hamming weight {weight}: {prob_of_equal_weight_in_origin_result} -> {prob_of_equal_weight_in_final_result}")

Hamming weight 0: 0.6439369463124412 -> 0.6439369463124444
Hamming weight 1: 0.3127480202331405 -> 0.31274802023314174
Hamming weight 2: 0.0418728319686951 -> 0.04187283196869525
Hamming weight 3: 0.0014420986893641766 -> 0.0014420986893641824
Hamming weight 4: 1.0279635909130541e-07 -> 1.027963590913069e-07
